In [1]:
#from wit import Wit
import pandas as pd
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

%matplotlib inline

# 1.Selected_comments_with_witai_sentiment_results
## import comment with side effects for query

In [2]:
SE_tem_l400 = pd.read_csv("wit_ai_sentment/SE_tem_l400.csv",index_col=0,header=None)[1]
SE_tem_l400 = SE_tem_l400.iloc[3:].astype(str)
idxs = list(SE_tem_l400.index)

In [3]:
first_quest2 = pd.read_csv("wit_ai_sentment/SE_tem_l400_first_request.ccv",index_col=0)

In [4]:
print("Total results:",len(first_quest2))
first_quest2.groupby('value').size()

Total results: 30689


value
negative    16765
neutral     12010
positive     1914
dtype: int64

In [5]:
for i_n in first_quest2[first_quest2['value'] == 'negative'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

I felt horrible (uncontrollable nausea, headaches, crazy mood changes, dry mouth) during the first month but I decided to stick with it since the side effects were &quot

I only have minor cramps which is normal and so far the only side effect is I feel hungry after taking the pill at night

I have not experienced any side effects

Really scary vertigo side effects, severe headache on the left side of my head

it seems I'm one of the few who didn't have any negative side effects from it.   Like I said, I had NO side effects.   Just because the side effects aren't there doesn't mean the pill isn't working

The only bothersome side effect has been some increased urination urges during the night

There are a few side effects, including out-of-nowhere jerking

Aside from slight appetite suppression, I've experienced no side effects

No negative side effects

No weight gain, no headache, no problems with sleep or other side effects related to SSRI



In [6]:
for i_n in first_quest2[first_quest2['value'] == 'neutral'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

I used Ortho-tri cyclen for 8 years, and had very few side effects

I have no side effects, VL is still undetectable, and CD4 is steady

That's keeping me sane and I am on week 4 overall: 16 days using medication the remainder trying to overcome side effects

no major side effects

We have had no side effects and we use it for major flare-ups as it is not intended for long term daily use

I have had next to no side effects

That being said, these type of medicines have different side effects for each individual, i.  there is no standard side effect

The only side effect I've had is the &quot

I'm still going to give it a try for another month to see if those side effects wear off.  I have no other side effects

I took the pill around 40 hours later, and experienced no side effects



In [7]:
for i_n in first_quest2[first_quest2['value'] == 'positive'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

I don't remember having side effects because I was so happy I got my cycle

Used it for 7 years, no side effects, works great to reduce my Blood Pressure

I took the 2 week trial and it worked great, no side effects, and I felt great, I went from 185-180 in the 2 weeks, I felt full right away, dehydrated so I was drinking ton's of water, craved no sweets, small portions filled me up fast, now I'm on the 7

I used Mobic for 1 year with great results for knee pain andno side effects

I was happy with that, and seeing no side effects, I went on to taking the next dose of 7.   The only side effects I had was an occasional dry mouth, which I battled by drinking more water, which is recommended with this drug anyways

I've finally found something to help me make it through the night without side effects

Another great thing about it is I don't have any side effects

Oh my side effects dreams I was on a cooking show, gained a little weight

Side effects are bad initially but now I am able to 

# 2. Train the model using only these comment with side effects 

In [8]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from wordcloud import WordCloud

from sklearn.svm import SVC

## only need to remove punctuation and stemize
stemmer = SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [11]:
## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

In [ ]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0)
df = rm_sym(df)


test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0)
test = rm_sym(test)

In [26]:
train = pd.concat([df.loc[first_quest2.index], first_quest2],1)
## Generate table of words with their counts
con_vec = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
X_train_p = con_vec.fit_transform(train['review'])
#y_train = train['rating_cate']


/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [27]:
negnornot = (train['value'] == "negative").astype(int)
negnornot.shape

(30689,)

In [28]:
X_train,X_test,y_train,y_test =  train_test_split(X_train_p,negnornot,test_size=0.25)

## Logistic regression

In [29]:
lr = LogisticRegression(penalty='l1',multi_class='auto',solver='saga')
#lr.fit(X,target_3.iloc[:,0])

cv_score = cross_val_score(lr,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
cv_score

array([0.68886861, 0.67961418, 0.6835245 ])

### RandomForestClassifier

In [30]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc_cv_score = cross_val_score(rfc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
rfc_cv_score

array([0.7632951 , 0.77111575, 0.7653806 ])

In [31]:
rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc.fit(X_train,y_train)
y_test_predict = rfc.predict(X_test)
accuracy_score(y_test,y_test_predict)

0.8118076371692949

In [32]:
feature_importances = pd.DataFrame(rfc.feature_importances_,con_vec.get_feature_names())
feature_importances.sort_values(0,ascending=False)

,0
bad,0.011099
effect,0.009980
headach,0.005513
horribl,0.005404
year,0.005260
work,0.005085
onli,0.004601
day,0.004482
week,0.004296
month,0.004151


### Gradient boosting classifier

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=300)
gbc_cv_score = cross_val_score(gbc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.69721064, 0.68847758, 0.69890511])

### SVM

#### poly

In [34]:
svm_poly = SVC(kernel='poly')
gbc_cv_score = cross_val_score(svm_poly,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


KeyboardInterrupt: 

#### linear

In [ ]:
svm_lin = SVC(kernel='linear')
gbc_cv_score = cross_val_score(svm_lin,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

## Using LSTM to predict whether negative

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
#import requests
%matplotlib inline
from sklearn.svm import SVC
from nltk.stem.snowball import SnowballStemmer

## remove special symbol

def clean_text(df_tem3,col):
    df_tem3[col] = df_tem3[col].str.replace("&#039;",'\'')
    df_tem3[col] = df_tem3[col].str.replace("\"","").str.lower()
    df_tem3[col] = df_tem3[col].str.replace( r"(\\r)|(\\n)|(\\t)|(\\f)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(&#039;)|(\d\s)|(\d)|(\/)","")
    df_tem3[col] = df_tem3[col].str.replace("\"","").str.lower()
    df_tem3[col] = df_tem3[col].str.replace( r"(\$)|(\-)|(\\)|(\s{2,})"," ")
    df_tem3[col].sample(1).iloc[0]

    stemmer = SnowballStemmer('english')
    df_tem3[col] = df_tem3[col].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split(" ")]))
    return df_tem3


np.random.seed(9)

In [10]:
import tensorflow as tf
import tensorflow 

#from tensorflow import tensorflow.keras

#from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional



# fix random seed for reproducibility

MAX_NB_WORDS = 500
max_review_length = 500
EMBEDDING_DIM = 160


In [11]:
first_quest3 = first_quest2.copy()
first_quest3.columns = ['review', 'confidence', 'value']
df_tem3 = clean_text(first_quest3,'review')


In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_tem3,test_size=0.2)

In [13]:
# Tokenize the data
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(train['review'])
train_sequences = tokenizer.texts_to_sequences(train['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])

In [14]:
# truncate and pad input sequences
X_train = sequence.pad_sequences(train_sequences, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_sequences, maxlen = max_review_length)

In [15]:
word_index = tokenizer.word_index

print(X_train.shape, '<-- shape of train_data ready for val/train split.')
print(X_test.shape, '<-- shape of final_test_data ready for fedding to network.')
print(len(tokenizer.word_index), '<-- Length of Word Index')

(24551, 500) <-- shape of train_data ready for val/train split.
(6138, 500) <-- shape of final_test_data ready for fedding to network.
6232 <-- Length of Word Index


In [16]:
y_train = (train['value'] == 'negative').astype(int)
y_test = (test['value'] == 'negative').astype(int)
print(y_train.shape, '<-- shape of train_data ready for val/train split.')
print(y_test.shape, '<-- shape of final_test_data ready for fedding to network.')

(24551,) <-- shape of train_data ready for val/train split.
(6138,) <-- shape of final_test_data ready for fedding to network.


In [89]:
nb_words  = min(MAX_NB_WORDS, len(word_index))
lstm_out = max_review_length

model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,input_length=max_review_length))
#model.add(Dropout(0.2))

## add conv using kernal No.32 and size 3x3, actiation='relu'(rm neg)
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPool1D(pool_size=2))
model.add(Bidirectional(LSTM(20, return_sequences=True)))
#model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(20)))
#model.add(Attention(max_review_length))
model.add(Dense(1, activation = 'sigmoid'))

## one-code mutiple categories targets use 'categorical_crossentropy' not 'binary_crossentropy'
model.compile(loss='binary_crossentropy',optimizer='adam',metrics =['accuracy'])

In [90]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 160)          80000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 40)           28960     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40)                9760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 41        
Total params: 118,761
Trainable params: 118,761
Non-trainable params: 0
_________________________________________________________________


In [91]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


## if 4 steps without decreasing of loss in valid set, stop the trainning
batch = 32 
epoch = 40

early_stopping = EarlyStopping(patience = 4)

LSTM_model = model.fit(X_train, y_train, batch_size=batch, epochs=epoch,
                       validation_split=0.2,callbacks=[early_stopping], shuffle = True)

best_score = min(LSTM_model.history['val_loss'])

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 19640 samples, validate on 4911 samples
Epoch 1/40
19640/19640 [==============================] - 493s 25ms/step - loss: 0.4132 - acc: 0.8116 - val_loss: 0.3230 - val_acc: 0.8621
Epoch 2/40
19640/19640 [==============================] - 490s 25ms/step - loss: 0.3153 - acc: 0.8706 - val_loss: 0.3008 - val_acc: 0.8742
Epoch 3/40
19640/19640 [==============================] - 488s 25ms/step - loss: 0.2877 - acc: 0.8791 - val_loss: 0.2970 - val_acc: 0.8776
Epoch 4/40
19640/19640 [==============================] - 488s 25ms/step - loss: 0.2674 - acc: 0.8903 - val_loss: 0.2903 - val_acc: 0.8850
Epoch 5/40
19640/19640 [==============================] - 488s 25ms/step - loss: 0.2522 - acc: 0.8968 - val_loss: 0.2910 - val_acc: 0.8833
Epoch 6/40
19640/19640 [==============================] - 488s 25ms/step - loss: 0.2372 - acc: 0.9032 - val_loss: 0.2876 - val_acc: 0.8856
Epoch 7/40
19640/19640 [==============================] - 487s 25ms/step - loss: 0.2232 - acc: 0.9090 - val_loss: 0.2

In [92]:
LSTM_model_history = pd.DataFrame(LSTM_model.history)
file_name = "Wit_ai_" + "bilstm_2l_20neu.csv"
LSTM_model_history.to_csv('./model_report/' + file_name)

In [93]:
model.save('./model_report/Wit_ai_bilstm_2l_20neu.h5')

In [94]:
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('./model_report/Wit_ai_bilstm_2l_20neu.h5')

accr = model.evaluate(X_test,y_test, batch_size = 100)
print(accr)
print("both 1 and 2 layers of directional LSTM works similiarly")

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6138/6138 [==============================] - 14s 2ms/step
[0.34674848232062644, 0.8763440850310079]
both 1 and 2 layers of directional LSTM works similiarly


In [95]:
unkn_data = pd.read_csv("web_scrap.csv",index_col=0)
unkn_data2 = unkn_data[['3','9','10']]
unkn_data2.columns = ['SE','SE_comments','comments']
unkn_data2 = unkn_data2.drop_duplicates()
unkn_data2['SE_YN'] = ((unkn_data2['SE'] != ' No Side Effects') & (unkn_data2['SE'] != ' Mild Side Effects')).astype(int)
unkn_data2.groupby('SE_YN').size()

SE_YN
0    2552
1    1606
dtype: int64

In [96]:
#unkn_data2[(unkn_data2['SE'] != ' No Side Effects') & (unkn_data2['SE'] != ' Mild Side Effects')]

In [97]:
unkn_test = clean_text(unkn_data2,'comments')

unkn_test_sequences = tokenizer.texts_to_sequences(unkn_test['comments'])
X_unkn_test = sequence.pad_sequences(unkn_test_sequences, maxlen = max_review_length)
y_unkn_test = unkn_data2['SE_YN']

In [99]:
unkn_test_accr = model.evaluate(X_unkn_test,y_unkn_test, batch_size = 100)
print(unkn_test_accr)

4158/4158 [==============================] - 9s 2ms/step
[1.9235212075819241, 0.5526695493868534]


In [100]:
from sklearn.metrics import confusion_matrix

In [101]:
y_unkn_test_pred = model.predict_classes(X_unkn_test)

In [102]:
confusion_matrix(y_unkn_test,y_unkn_test_pred)

array([[1409, 1143],
       [ 717,  889]])

In [86]:
unkn_test['pred_SE_YN'] = y_unkn_test_pred
for a in unkn_test[(unkn_test['pred_SE_YN'] == 1)]['SE_comments'].sample(10):
    print(a+'\n')

 No side effects.

 drowsiness, urinary retention may have caused dry mouth but not as bad as using ditropan does

 There were many!  Enough that I stopped taking the Diovan and went back to the higher dose of Verapamil...  I am over 50 and menopausal so weight gain is an issue anyway.  Diovan causes substantial increase in hunger.  Substantial.  Not just an urge to eat but hungry to the point of discomfort.  I tried eating carrots and celery, drinking lots of water and milk, but still I gained weight.  I also lost virtually ALL of my sex drive.  

 upset stomach, low appetite, metallic taste, tremor

 Nausea, bad dreams, very prone to sunburn. For the antibiotic to be effective I need to take a relatively large dose for my size (50kgs). Have to take anti-nausea medication while on the course of antibiotic otherwise tend to lose weight.

 Dry mouth and shaky hands . Shakiness subsided in 3-4 days  

 none

 sickness for 4 days, numb arm for that period of time, couldn't dress, couldn't

In [87]:
unkn_test['pred_SE_YN'] = y_unkn_test_pred
for a in unkn_test[(unkn_test['pred_SE_YN'] == 0)]['SE_comments'].sample(10):
    print(a+'\n')

 none. i found this drug very effectivewith no side effects what so ever.  i am about to lower the dose to 5mg as a maintenance treatment.  my blood tests proved the effectivness of Crestor.  from  6.5 down to 3.6

 I dream a little more.  

 Drowsiness which was not a major issue for me. phase of depression each time i stopped taking the pill. I didn't realize what was causing it until after the last couple of times. I mean a serious phase of depression which was new to me since i have never experienced depression before

 Cognitive deficits, memory problems, word retrieval difficulties, inability to experience complex emotions, lack of motivation, true lack of ability to absorb written information. 

 Severe muscle pain and weakness

 initially experienced digestive discomfort, diarrhea, agitation and insomnia, dry mouth, chills and sweats, and headache; all mild and disappeared after about 2 weeks.

 I have found none besides it took 3 months to fully kick in. This birth control is 

In [88]:
unkn_test#.groupby('SE').size()

,SE,SE_comments,comments,SE_YN,pred_SE_YN
0,Severe Side Effects,A very uncomfortable inner restlessness was t...,i was prescrib abilifi mgdaili to assist in m...,1,0
1,Moderate Side Effects,"I became drowsy, however, with adequate sleep...",the abilifi decreas the need for daili klonop...,1,0
2,No Side Effects,no side effects have been noticed,i take one pill of each st thing in the ami a...,0,1
3,Extremely Severe Side Effects,but it caused memory loss and again an incid...,i am onli take ativan & get psycholog therapi...,1,1
4,Severe Side Effects,Headache first morning at 4AM that was reliev...,not much to tell i was just start treatment w...,1,0
5,No Side Effects,None so far.,my doctor ad abilifi to my mg of cymbalta bec...,0,0
8,Mild Side Effects,"Dryness, slight redness and a small amount of...",the combin of clindamycin and benzoyl peroxid...,0,0
9,No Side Effects,I tolerated the medication very well. I didn...,i was requir to take accol for three month to...,0,0
10,No Side Effects,none,i have alway been normal weight but after men...,0,1
11,No Side Effects,None I am aware of.,taken onc daili in combin with hyzaar my cond...,0,0
